In [13]:
import pandas as pd
from itertools import permutations
import math
import os
import csv
import time

In [14]:
cargo_capacity_psv = 100
psv_speed = 7 #7 for strøm, 10 på resten
max_platforms_in_one_voyage = 7

## Reads input data

In [15]:
demand = pd.read_csv('clustering/output_platforms_demand.csv', header=0, delimiter=';')
distances = pd.read_csv('clustering/output_distance_matrix_kmeans.csv', header=0, delimiter=';', index_col='from/to')

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['DUS'] + demand['platform'].tolist() + ['DUS']  # Add 'DUS' as start and end platforms

## Algorithm for generating the shortest routes

In [16]:
shortest_routes_dict = {}

def generate_routes(demand, distances):
    cargo_capacity = cargo_capacity_psv
    max_platforms = max_platforms_in_one_voyage + 2

    def dp(platform, cargo_remaining, route, visited):
        if cargo_remaining < 0:
            return
        if len(route) > max_platforms:
            return
        if platform == 'DUS' and len(route) > 2:
            total_demand = sum(platforms_demand[p] for p in route[1:-1])
            if total_demand <= cargo_capacity:
                key = tuple(sorted(set(route)))
                total_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
                if key not in shortest_routes_dict or total_distance < shortest_routes_dict[key][1]:
                    duration_sailing = round((total_distance / psv_speed), 2)
                    duration_lossing = round(((total_demand * 1.389) / psv_speed), 2)
                    duration_sailing = round(duration_sailing, 2)
                    duration_lossing = round(duration_lossing, 2)
                    shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            return

        # Check if the current route is dominated
        current_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
        current_demand = sum(platforms_demand[p] for p in route[1:-1])
        
        # Check for dominance in existing routes
        for key, (existing_route, existing_distance, existing_demand, _, _) in shortest_routes_dict.items():
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return
        
        # Check for dominance in subsequent routes starting with the same platforms
        for existing_route in dominated_routes:
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return

        for next_platform in platforms_demand.keys():
            if next_platform != platform and next_platform not in visited:
                try:
                    distance_to_next = distances.loc[platform, next_platform]
                    new_cargo_remaining = cargo_remaining - platforms_demand[next_platform]
                    dp(next_platform, new_cargo_remaining, route + [next_platform], visited.union({next_platform}))
                except KeyError:
                    print("KeyError occurred for platform:", next_platform)
                    continue

    for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):
        for route_combination in permutations(platforms_demand.keys(), r - 2):
            route = ['DUS'] + list(route_combination) + ['DUS']
            dp('DUS', cargo_capacity, route, {'DUS'})

            # Add the current route to the dominated routes set
            dominated_routes.add(tuple(route))

    return shortest_routes_dict

# Keep track of dominated routes to skip future routes starting with them
dominated_routes = set()
shortest_routes_dict = generate_routes(demand, distances)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {round(distance,2)}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print(len(shortest_routes_dict))

Shortest Route: ['DUS', 'DABGRAHDA', 'DUS'], Total Distance: 235.0, Total Demand: 68.5, Duration sailing: 33.57, Duration lossing: 13.59
Shortest Route: ['DUS', 'DRA', 'DUS'], Total Distance: 205.36, Total Demand: 14.0, Duration sailing: 29.34, Duration lossing: 2.78
Shortest Route: ['DUS', 'DSAJSF', 'DUS'], Total Distance: 183.8, Total Demand: 61.75, Duration sailing: 26.26, Duration lossing: 12.25
Shortest Route: ['DUS', 'GKRSLASLB', 'DUS'], Total Distance: 246.84, Total Demand: 18.0, Duration sailing: 35.26, Duration lossing: 3.57
Shortest Route: ['DUS', 'GUD', 'DUS'], Total Distance: 226.76, Total Demand: 18.0, Duration sailing: 32.39, Duration lossing: 3.57
Shortest Route: ['DUS', 'TEB', 'DUS'], Total Distance: 250.32, Total Demand: 14.0, Duration sailing: 35.76, Duration lossing: 2.78
Shortest Route: ['DUS', 'DABGRAHDA', 'DRA', 'DUS'], Total Distance: 309.07, Total Demand: 82.5, Duration sailing: 44.15, Duration lossing: 16.37
Shortest Route: ['DUS', 'DABGRAHDA', 'GKRSLASLB', 'DU

## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 858 routes

In [17]:
routes_only = [route for route, _, _, _, _ in shortest_routes_dict.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv('generated_datafiles_cluster6/distances.csv', index=False)
df_demand.to_csv('generated_datafiles_cluster6/demand.csv', index=False)
df_duration_sailing.to_csv('generated_datafiles_cluster6/duration_sailing.csv', index=False)
df_duration_lossing.to_csv('generated_datafiles_cluster6/duration_lossing.csv', index=False)
route_file = "generated_datafiles_cluster6/routes.csv"


def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)

## Creating the reduced set of routes

In [21]:
'''
import csv

def contains_required_routes(route, required_routes):
    return all(required_route in route for required_route in required_routes)

def main():
    hda_additional_routes1 = ["HDA", "SLB"]
    hda_additional_routes2 = ["HDA", "SLA"]
    hda_additional_routes3 = ["HDA", "DRA"]
    hda_additional_routes4 = ["HDA", "TEB"]
    
    dra_additional_routes1 = ["DRA", "DAB"]
    dra_additional_routes2 = ["DRA", "GRA"]

    gkr_additional_routes1 = ["GKR", "HDA"]

    teb_additional_routes1 = ["TEB", "DAB"]
    teb_additional_routes2 = ["TEB", "GRA"]

    sla_additional_routes1 = ["SLA", "DAB"]
    sla_additional_routes2 = ["SLA", "GRA"]

    slb_additional_routes1 = ["SLB", "DAB"]
    slb_additional_routes2 = ["SLB", "GRA"]

    routes_with_hda_slb = []
    routes_with_hda_sla = []
    routes_with_hda_dra = []
    routes_with_hda_teb = []

    routes_with_dra_dab = []
    routes_with_dra_gra = []
    routes_with_gkr_hda = []
    routes_with_teb_dab = []

    routes_with_teb_gra = []
    routes_with_sla_dab = []
    routes_with_sla_gra = []
    routes_with_slb_dab = []
    routes_with_slb_gra = []

    with open('generated_datafiles_allroutes/routes.csv', 'r') as file:
        csv_reader = csv.reader(file)
        for row_number, row in enumerate(csv_reader, start=1):
            if contains_required_routes(row, hda_additional_routes1):
                routes_with_hda_slb.append(row_number)
            if contains_required_routes(row, hda_additional_routes2):
                routes_with_hda_sla.append(row_number)
            if contains_required_routes(row, hda_additional_routes3):
                routes_with_hda_dra.append(row_number)
            if contains_required_routes(row, hda_additional_routes4):
                routes_with_hda_teb.append(row_number)
            if contains_required_routes(row, dra_additional_routes1):
                routes_with_dra_dab.append(row_number)
            if contains_required_routes(row, dra_additional_routes2):
                routes_with_dra_gra.append(row_number)

            if contains_required_routes(row, gkr_additional_routes1):
                routes_with_gkr_hda.append(row_number)

            if contains_required_routes(row, teb_additional_routes1):
                routes_with_teb_dab.append(row_number)
            if contains_required_routes(row, teb_additional_routes2):
                routes_with_teb_gra.append(row_number)
            if contains_required_routes(row, sla_additional_routes1):
                routes_with_sla_dab.append(row_number)
            if contains_required_routes(row, sla_additional_routes2):
                routes_with_sla_gra.append(row_number)
            if contains_required_routes(row, slb_additional_routes1):
                routes_with_slb_dab.append(row_number)
            if contains_required_routes(row, slb_additional_routes2):
                routes_with_slb_gra.append(row_number)
            
    # Finding rows where all combinations occur
    # Combine all lists into one
    all_routes = (
        routes_with_hda_slb + 
        routes_with_hda_sla +
        routes_with_hda_dra +
        routes_with_hda_teb +
        routes_with_dra_dab +
        routes_with_dra_gra +
        routes_with_gkr_hda +
        routes_with_teb_dab +
        routes_with_teb_gra +
        routes_with_sla_dab +
        routes_with_sla_gra +
        routes_with_slb_dab +
        routes_with_slb_gra
    )
    
    # Convert the combined list into a set to remove duplicates
    unique_routes = set(all_routes)

    # Convert the set back into a list if needed
    unique_routes_list = list(unique_routes)
    print((sorted(unique_routes_list)))
    print(len(unique_routes_list))

    def filter_routes(input_file, output_file, exclude_indices):
        with open(input_file, 'r', newline='') as f_in, \
            open(output_file, 'w', newline='') as f_out:
            reader = csv.reader(f_in)
            writer = csv.writer(f_out)

            for i, row in enumerate(reader):
                if i + 1 not in exclude_indices:  # Adjusting for 1-based indexing
                    writer.writerow(row)

    def filter_otherfiles(input_file, output_file, exclude_indices):
        with open(input_file, 'r', newline='') as f_in, \
            open(output_file, 'w', newline='') as f_out:
            reader = csv.reader(f_in)
            writer = csv.writer(f_out)

            for i, row in enumerate(reader):
                if i not in exclude_indices:  
                    writer.writerow(row)

    # Example usage
    input_file = 'generated_datafiles_allroutes/routes.csv'
    output_file = 'generated_datafiles_reducedroutes/routes.csv'

    input_file2 = 'generated_datafiles_allroutes/demand.csv'
    output_file2 = 'generated_datafiles_reducedroutes/demand.csv'

    input_file3 = 'generated_datafiles_allroutes/distances.csv'
    output_file3 = 'generated_datafiles_reducedroutes/distances.csv'

    input_file4 = 'generated_datafiles_allroutes/duration_lossing.csv'
    output_file4 = 'generated_datafiles_reducedroutes/duration_lossing.csv'

    input_file5 = 'generated_datafiles_allroutes/duration_sailing.csv'
    output_file5 = 'generated_datafiles_reducedroutes/duration_sailing.csv'

    filter_routes(input_file, output_file, unique_routes_list)
    filter_otherfiles(input_file2, output_file2, unique_routes_list)
    filter_otherfiles(input_file3, output_file3, unique_routes_list)
    filter_otherfiles(input_file4, output_file4, unique_routes_list)
    filter_otherfiles(input_file5, output_file5, unique_routes_list)

if __name__ == "__main__":
    main()
'''


[27, 61, 62, 63, 72, 105, 106, 107, 115, 122, 128, 133, 137, 141, 142, 143, 169, 170, 171, 190, 191, 192, 205, 206, 207, 215, 216, 217, 221, 222, 223, 227, 228, 229, 237, 242, 247, 251, 252, 253, 264, 265, 278, 279, 280, 288, 289, 290, 297, 298, 299, 303, 304, 305, 309, 314, 322, 326, 327, 328, 334, 339, 343, 347, 348, 349, 354, 358, 362, 363, 364, 368, 372, 373, 374, 378, 379, 380, 384, 385, 386, 399, 400, 401, 411, 412, 413, 420, 421, 425, 426, 427, 431, 432, 433, 444, 445, 446, 454, 455, 456, 460, 461, 462, 466, 467, 468, 474, 475, 476, 480, 481, 482, 486, 487, 488, 490, 491, 492, 496, 497, 498, 500, 501, 502, 504, 511, 512, 519, 520, 524, 525, 526, 530, 531, 532, 537, 541, 545, 546, 550, 551, 552, 554, 555, 559, 560, 561, 563, 564, 565, 567, 569, 573, 577, 578, 579, 583, 587, 588, 589, 594, 595, 596, 600, 601, 602, 606, 610, 611, 612, 619, 620, 624, 625, 626, 630, 631, 632, 637, 638, 642, 643, 644, 648, 649, 650, 652, 653, 654, 658, 659, 660, 662, 663, 664, 666, 671, 672, 673, 678,